# Embedding Based Retrieval: Text

# About

In this notebook we will do follows:
- Create a text corpus by using captions
- Use existing embedding model `sentence-transformers/all-MiniLM-L6-v2`
- Explore the retrieval results of the model 

# About the Embedding model

About the model `sentence-transformers/all-MiniLM-L6-v2`


From the [Docs](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

It maps sentences & paragraphs to a `384` dimensional dense vector space and can be used for tasks like clustering or semantic search.

The project aims to train sentence embedding models on very large sentence level datasets using a self-supervised contrastive learning objective.    

We fine-tuned in on a 1B sentence pairs dataset. 

We use a contrastive learning objective: given a sentence from the pair, the model should predict which out of a set of randomly sampled other sentences, was actually paired with it in our dataset.






| **Dataset**                                                                                                                      | **Number of training tuples** |
| -------------------------------------------------------------------------------------------------------------------------------- | ----------------------------- |
| [Reddit comments (2015-2018)](https://github.com/PolyAI-LDN/conversational-datasets/tree/master/reddit)                          | 726,484,430                   |
| [S2ORC](https://github.com/allenai/s2orc) Citation pairs (Abstracts)                                                             | 116,288,806                   |
| [WikiAnswers](https://github.com/afader/oqa#wikianswers-corpus) Duplicate question pairs                                         | 77,427,422                    |
| [PAQ](https://github.com/facebookresearch/PAQ) (Question, Answer) pairs                                                          | 64,371,441                    |
| [S2ORC](https://github.com/allenai/s2orc) Citation pairs (Titles)                                                                | 52,603,982                    |
| [S2ORC](https://github.com/allenai/s2orc) (Title, Abstract)                                                                      | 41,769,185                    |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) (Title, Body) pairs                 | 25,316,456                    |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) (Title+Body, Answer) pairs          | 21,396,559                    |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) (Title, Answer) pairs               | 21,396,559                    |
| [MS MARCO](https://microsoft.github.io/msmarco/) triplets                                                                        | 9,144,553                     |
| [GOOAQ: Open Question Answering with Diverse Answer Types](https://github.com/allenai/gooaq)                                     | 3,012,496                     |
| [Yahoo Answers](https://www.kaggle.com/soumikrakshit/yahoo-answers-dataset) (Title, Answer)                                      | 1,198,260                     |
| [Code Search](https://huggingface.co/datasets/code_search_net)                                                                   | 1,151,414                     |
| [COCO](https://cocodataset.org/#home) Image captions                                                                             | 828,395                       |
| [SPECTER](https://github.com/allenai/specter) citation triplets                                                                  | 684,100                       |
| [Yahoo Answers](https://www.kaggle.com/soumikrakshit/yahoo-answers-dataset) (Question, Answer)                                   | 681,164                       |
| [Yahoo Answers](https://www.kaggle.com/soumikrakshit/yahoo-answers-dataset) (Title, Question)                                    | 659,896                       |
| [SearchQA](https://huggingface.co/datasets/search_qa)                                                                            | 582,261                       |
| [Eli5](https://huggingface.co/datasets/eli5)                                                                                     | 325,475                       |
| [Flickr 30k](https://shannon.cs.illinois.edu/DenotationGraph/)                                                                   | 317,695                       |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) Duplicate questions (titles)        | 304,525                       |
| AllNLI ([SNLI](https://nlp.stanford.edu/projects/snli/) and [MultiNLI](https://cims.nyu.edu/~sbowman/multinli/)                  | 277,230                       |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) Duplicate questions (bodies)        | 250,519                       |
| [Stack Exchange](https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_xml) Duplicate questions (titles+bodies) | 250,460                       |
| [Sentence Compression](https://github.com/google-research-datasets/sentence-compression)                                         | 180,000                       |
| [Wikihow](https://github.com/pvl/wikihow_pairs_dataset)                                                                          | 128,542                       |
| [Altlex](https://github.com/chridey/altlex/)                                                                                     | 112,696                       |
| [Quora Question Triplets](https://quoradata.quora.com/First-Quora-Dataset-Release-Question-Pairs)                                | 103,663                       |
| [Simple Wikipedia](https://cs.pomona.edu/~dkauchak/simplification/)                                                              | 102,225                       |
| [Natural Questions (NQ)](https://ai.google.com/research/NaturalQuestions)                                                        | 100,231                       |
| [SQuAD2.0](https://rajpurkar.github.io/SQuAD-explorer/)                                                                          | 87,599                        |
| [TriviaQA](https://huggingface.co/datasets/trivia_qa)                                                                            | 73,346                        |
| **Total**                                                                                                                        | **1,170,060,424**             |

Other models listed in this page are also great candidates

https://www.sbert.net/docs/pretrained_models.html#model-overview

## Setup

In [1]:
from sentence_transformers import SentenceTransformer

import datasets
import rich
from IPython.display import Image, JSON
from IPython.core.display import HTML
import numpy as np

from transformers import AutoTokenizer
import ipywidgets as widgets
from ipywidgets import interact
import ipyplot
import os

In [2]:
dset = datasets.load_from_disk("../data/processed")

In [3]:
dset

Dataset({
    features: ['photo_id', 'photo_url', 'photo_image_url', 'photo_submitted_at', 'photo_featured', 'photo_width', 'photo_height', 'photo_aspect_ratio', 'photo_description', 'photographer_username', 'photographer_first_name', 'photographer_last_name', 'exif_camera_make', 'exif_camera_model', 'exif_iso', 'exif_aperture_value', 'exif_focal_length', 'exif_exposure_time', 'photo_location_name', 'photo_location_latitude', 'photo_location_longitude', 'photo_location_country', 'photo_location_city', 'stats_views', 'stats_downloads', 'ai_description', 'ai_primary_landmark_name', 'ai_primary_landmark_latitude', 'ai_primary_landmark_longitude', 'ai_primary_landmark_confidence', 'blur_hash', 'description_final', 'image', 'image_path_local'],
    num_rows: 24992
})

In [4]:
rich.print ( dset[0] )

{
    'photo_id': 'XMyPniM9LF0',
    'photo_url': 'https://unsplash.com/photos/XMyPniM9LF0',
    'photo_image_url': 'https://images.unsplash.com/uploads/14119492946973137ce46/f1f2ebf3',
    'photo_submitted_at': '2014-09-29 00:08:38.594364',
    'photo_featured': 't',
    'photo_width': 4272,
    'photo_height': 2848,
    'photo_aspect_ratio': 1.5,
    'photo_description': 'Woman exploring a forest',
    'photographer_username': 'michellespencer77',
    'photographer_first_name': 'Michelle',
    'photographer_last_name': 'Spencer',
    'exif_camera_make': 'Canon',
    'exif_camera_model': 'Canon EOS REBEL T3',
    'exif_iso': 400.0,
    'exif_aperture_value': '1.8',
    'exif_focal_length': '50.0',
    'exif_exposure_time': '1/100',
    'photo_location_name': None,
    'photo_location_latitude': None,
    'photo_location_longitude': None,
    'photo_location_country': None,
    'photo_location_city': None,
    'stats_views': 2375421,
    'stats_downloads': 6967,
    'ai_description': 'woman walking in the middle of forest',
    'ai_primary_landmark_name': None,
    'ai_primary_landmark_latitude': None,
    'ai_primary_landmark_longitude': None,
    'ai_primary_landmark_confidence': None,
    'blur_hash': 'L56bVcRRIWMh.gVunlS4SMbsRRxr',
    'description_final': 'Woman exploring a forest',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427 at 0x7F257129E950>,
    'image_path_local': '../data/raw/images/XMyPniM9LF0.jpg'
}

## Model

In [5]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

In [6]:

# Initialize retriever with SentenceTransformer model 
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Tokenizer

In [46]:
text = "A man and his pet dog Perrito playdingd catch in the beach."

In [47]:
resp = model.encode(text, output_value=None)
rich.print(resp)

{
    'input_ids': tensor([  101,  1037,  2158,  1998,  2010,  9004,  3899,  2566, 28414,  2377,
         4667,  2094,  4608,  1999,  1996,  3509,  1012,   102],
       device='cuda:0'),
    'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
    'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
    'token_embeddings': tensor([[-0.0677,  0.0513,  0.4768,  ...,  0.3576, -0.3906, -0.1148],
        [-0.1968, -0.0721, -0.3324,  ...,  0.5298, -0.2774,  0.5287],
        [ 0.0858,  0.4833, -0.3329,  ..., -0.3483, -0.4282, -0.2897],
        ...,
        [ 0.7096,  0.1959,  0.1379,  ...,  0.0101, -0.1908, -0.1463],
        [ 0.0943,  0.1842,  0.1608,  ...,  0.4050, -0.3372,  0.2188],
        [ 0.1566,  0.1167,  0.1975,  ...,  0.3038, -0.2227,  0.2381]],
       device='cuda:0'),
    'sentence_embedding': tensor([ 2.7245e-02,  8.5115e-04,  4.1074e-02, -6.1840e-02, -4.6796e-02,
        -3.2752e-03,  6.4249e-02, -3.3732e-02,  3.3450e-02,  1.3065e-02,
         8.5493e-02, -4.2203e-02,  2.4165e-02,  5.2184e-02,  7.6807e-02,
         3.9865e-02, -5.7506e-02,  2.4284e-02,  8.2526e-02, -3.1512e-02,
        -6.5045e-02,  8.3287e-03, -1.0168e-01, -5.3761e-02, -1.8875e-01,
        -3.0703e-02,  7.5860e-02, -2.2065e-02, -8.8094e-03, -1.9613e-02,
        -1.9956e-02, -1.8503e-02,  6.7821e-03,  1.7351e-03, -4.2387e-02,
        -3.5774e-02, -4.4546e-02, -3.3200e-02, -1.3204e-02, -2.5218e-04,
        -2.1067e-02, -1.3864e-02,  8.8424e-02,  2.3969e-02, -4.1921e-02,
        -1.2925e-02, -1.5744e-02, -4.8291e-02,  8.3471e-02, -3.5383e-02,
        -4.0611e-02,  2.0836e-02, -4.1337e-03, -2.6046e-02, -7.0935e-02,
        -4.3526e-02, -8.0206e-02,  3.2333e-02, -8.0339e-02, -1.4097e-02,
         5.7811e-02,  2.1070e-02,  4.7009e-02,  1.0136e-02,  5.0523e-02,
        -4.3152e-02,  1.2582e-02,  2.0966e-02, -1.4002e-02, -2.4897e-02,
         1.5817e-02, -1.2991e-02,  7.8685e-02, -5.7955e-02, -1.9291e-02,
         2.6861e-03, -7.0163e-02,  3.8303e-03,  1.1840e-02, -6.3685e-02,
        -5.4816e-02,  1.6686e-03, -7.1812e-02,  1.8636e-02,  8.1678e-02,
         3.1124e-02, -2.3793e-02,  1.9062e-02,  1.8774e-02,  3.5100e-02,
        -8.2608e-02, -6.7733e-02, -5.0751e-02, -3.9201e-02, -6.1961e-02,
         4.2451e-02, -3.8355e-02,  6.1236e-03, -2.9451e-02,  4.0979e-03,
         3.8161e-02,  6.1894e-02, -4.5156e-02, -8.8139e-04,  1.5279e-01,
         4.4970e-02,  3.9380e-03,  5.9288e-02,  5.6811e-02,  6.3213e-02,
        -5.9248e-02,  8.1122e-02, -1.4986e-02, -3.3264e-03,  5.6281e-02,
         1.0469e-01, -2.2121e-02, -6.2191e-02, -1.7965e-02, -1.0685e-01,
         1.1008e-02,  3.6424e-02, -3.8871e-02,  3.3848e-02,  5.2330e-02,
         3.2203e-02,  3.8796e-02, -3.2964e-33,  2.6144e-02, -8.3933e-03,
        -4.7975e-02,  1.7831e-02, -7.7705e-03,  2.3873e-02, -5.4458e-02,
        -3.1801e-02, -3.4542e-02, -2.6412e-02, -2.8960e-02, -5.7436e-03,
        -3.6600e-02,  1.8724e-04,  4.3041e-02, -7.2709e-02,  3.3226e-02,
        -6.4818e-02,  3.5052e-02, -3.7997e-02, -2.1842e-02, -7.1258e-03,
        -4.4505e-02, -4.2587e-02, -1.1228e-02,  8.2499e-02, -6.4990e-03,
        -4.5247e-02,  3.6797e-02, -9.9252e-03, -2.2450e-02, -9.3727e-03,
         3.2621e-02, -3.0249e-02, -6.1823e-03, -2.2296e-02, -6.8490e-03,
        -4.0892e-02, -3.6306e-02,  5.1262e-02,  1.2915e-02,  7.6723e-03,
         1.4748e-02, -2.4131e-02, -8.1759e-02, -1.3676e-01, -2.8145e-02,
         6.4891e-02,  1.8963e-02, -5.3716e-02,  1.5972e-02, -2.5845e-02,
         4.5339e-02, -4.3962e-02, -4.5941e-02,  3.5499e-02,  5.3470e-02,
        -1.4657e-02, -6.0126e-02,  1.2750e-01,  1.0249e-01,  6.4104e-03,
         5.3865e-02,  3.9636e-02,  2.3959e-03, -5.0817e-02,  4.1301e-02,
         5.6234e-02,  5.6352e-03,  1.2783e-02,  7.5478e-03, -4.4354e-03,
         1.1062e-01, -4.0939e-02,  3.5345e-02, -1.9199e-02,  1.4538e-02,
         2.5399e-02,  4.6266e-02, -3.5740e-02,  1.2718e-02,  1.9636e-02,
         1.0668e-02,  1.9186e-02

number of tokens if based on whitespace

In [48]:
len(text)

59

number of tokens according to the model

In [49]:
resp['input_ids'].shape 

torch.Size([18])

each token has an embedding

In [50]:
resp['token_embeddings'].shape


torch.Size([18, 384])

All the token are pooled to generate an embedding

In [51]:
resp['sentence_embedding'].shape

torch.Size([384])

### Explore the tokenizer more

In [52]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


the number of unique words in the tokenizer

In [53]:
tokenizer.vocab_size

30522

In [56]:
resp = tokenizer(text)
resp

{'input_ids': [101, 1037, 2158, 1998, 2010, 9004, 3899, 2566, 28414, 2377, 4667, 2094, 4608, 1999, 1996, 3509, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [55]:
tokens = tokenizer.tokenize(text,add_special_tokens=True)
rich.print ( tokens)

[
    '[CLS]',
    'a',
    'man',
    'and',
    'his',
    'pet',
    'dog',
    'per',
    '##rito',
    'play',
    '##ding',
    '##d',
    'catch',
    'in',
    'the',
    'beach',
    '.',
    '[SEP]'
]

In [57]:
decoded_string = tokenizer.decode(resp['input_ids'])
decoded_string

'[CLS] a man and his pet dog perrito playdingd catch in the beach. [SEP]'

What happens if the input query is longer than the query

In [ ]:
# movie text of Puss in Boots https://en.wikipedia.org/wiki/Puss_in_Boots:_The_Last_Wish
text = f"""

Puss in Boots: The Last Wish is a 2022 American computer-animated adventure film produced by DreamWorks Animation and distributed by Universal Pictures. 

The sequel to the spin-off film Puss in Boots (2011) and the sixth installment in the Shrek franchise, the film was directed by Joel Crawford and co-directed by Januel Mercado. 

Based on the character from Shrek 2 (2004) and inspired by the eponymous fairy tale, the film's screenplay was written by Paul Fisher and Tommy Swerdlow, with a story by Swerdlow and Tom Wheeler (the latter of whom wrote the 2011 film). 

The voice cast of Puss in Boots: The Last Wish includes Antonio Banderas and Salma Hayek Pinault reprising their respective roles as the titular character and Kitty Softpaws, and are joined by Harvey Guillén, Florence Pugh, Olivia Colman, Ray Winstone, Samson Kayo, John Mulaney, Wagner Moura, Da'Vine Joy Randolph, and Anthony Mendez, who voice new characters introduced in the film. 

In the film, after the events of Shrek Forever After (2010), Puss in Boots teams up with Kitty and Perrito (Guillén) to find the mystical Last Wish for the Wishing Star to restore the first eight of his nine lives, before Goldilocks and her Three Bears Crime Family (Pugh, Winstone, Colman, and Kayo), and "Big" Jack Horner (Mulaney) do, while attempting to avoid a mysterious hooded wolf (Moura).


Plans for a sequel to Puss in Boots began in November 2012, when executive producer Guillermo del Toro shared plans to take the titular character on an adventure to a "very exotic locale". Work on a sequel began in April 2014, according to Banderas. After being stuck in development hell, the project was revived in November 2018, with Illumination founder and CEO Chris Meledandri confirmed to be an executive producer. It was announced that the film would be helmed by Bob Persichetti, the head of story of the first film and one of the three directors of Sony Pictures Animation's Spider-Man: Into the Spider-Verse (2018), in February 2019. Crawford was later announced as the new director in March 2021, along with Mercado. The story draws inspiration from Spaghetti Western films, with The Good, the Bad and the Ugly (1966) being cited as a particular influence. As with DreamWorks' previous film The Bad Guys (2022), the film has a stylized animation style, inspired by Spider-Man: Into the Spider-Verse. With new technology, the team was able to give the film a painterly style to resemble a fairy-tale story, rather than utilizing the realistic visual style of every previous installment of the Shrek franchise.

Puss in Boots: The Last Wish premiered at Lincoln Center in New York City on December 13, 2022, and was theatrically released in the United States on December 21, 2022, after being delayed due to restructuring at DreamWorks. The film received positive reviews from critics and was a commercial success, grossing $483 million worldwide on a production budget of $90–110 million, becoming the tenth-highest-grossing film of 2022. At the 95th Academy Awards it was nominated for Best Animated Feature, as well as receiving nominations at the Golden Globes, Critics' Choice Awards, and British Academy Film Awards.
"""



In [39]:
tokens = tokenizer.tokenize(text) 

In [40]:
len(tokens)

710

In [41]:
len(text),  tokenizer.model_max_length

(3194, 512)

In [42]:
print (" ".join( tokens ))

pu ##ss in boots : the last wish is a 202 ##2 american computer - animated adventure film produced by dream ##works animation and distributed by universal pictures . the sequel to the spin - off film pu ##ss in boots ( 2011 ) and the sixth installment in the sh ##rek franchise , the film was directed by joel crawford and co - directed by jan ##uel mer ##ca ##do . based on the character from sh ##rek 2 ( 2004 ) and inspired by the eponymous fairy tale , the film ' s screenplay was written by paul fisher and tommy sw ##er ##dlow , with a story by sw ##er ##dlow and tom wheeler ( the latter of whom wrote the 2011 film ) . the voice cast of pu ##ss in boots : the last wish includes antonio band ##eras and sal ##ma hay ##ek pin ##ault rep ##ris ##ing their respective roles as the titular character and kitty soft ##pa ##ws , and are joined by harvey gui ##llen , florence pu ##gh , olivia col ##man , ray winston ##e , samson kay ##o , john mu ##lane ##y , wagner mo ##ura , da ' vine joy rando

tokens after `512` position are skipped by the model.   


compute the embeddings of the entire corpus

# Embeddings

In [54]:
?model.encode

Signature:
model.encode(
    sentences: Union[str, List[str]],
    batch_size: int = 32,
    show_progress_bar: bool = None,
    output_value: str = 'sentence_embedding',
    convert_to_numpy: bool = True,
    convert_to_tensor: bool = False,
    device: str = None,
    normalize_embeddings: bool = False,
) -> Union[List[torch.Tensor], numpy.ndarray, torch.Tensor]
Docstring:
Computes sentence embeddings

:param sentences: the sentences to embed
:param batch_size: the batch size used for the computation
:param show_progress_bar: Output a progress bar when encode sentences
:param output_value:  Default sentence_embedding, to get sentence embeddings. Can be set to token_embeddings to get wordpiece token embeddings. Set to None, to get all output values
:param convert_to_numpy: If true, the output is a list of numpy vectors. Else, it is a list of pytorch tensors.
:param convert_to_tensor: If true, you get one large tensor as return. Overwrites any setting from convert_to_numpy
:param devic

In [56]:
corpus = model.encode(dset['description_final'], normalize_embeddings=True)

In [69]:
def find_results(query:str , k =5, explain=False,image_path_local_dir = "../data/raw/images"):
    
    # get embeddings of a query
    query_features = model.encode(query, normalize_embeddings=True)
    
    # compute cosine scores of the scores and all the other documents in the corpus
    doc_scores = query_features @ corpus.T
    
    # get index of top items
    top_items_idx = doc_scores.argsort()[-k:][::-1]


    tokens = tokenizer.tokenize(query) 
    degug_info = {
         "query_original":  query 
         , "query_processed" : query_features
         , "tokens": tokens
        , "doc_scores":  doc_scores 
        , "top_items":  top_items_idx 
    }
    
    

    display(HTML(f"<h4>Query: {query} </h4>"))
    
    if explain:
        rich.print (degug_info )
        
        
    
    images = []
    labels = []
    
    
  
    print(tokens)
    # Iterate over the top k results
    
    for idx, photo_data in enumerate( dset.select(top_items_idx)):

        
        doc_idx = top_items_idx[idx]
        
        photo_id = photo_data['photo_id']
        
        image_path_local = f"{image_path_local_dir}/{photo_id}.jpg"

        if os.path.exists(image_path_local):
            image_path = image_path_local
        else:
            image_path = photo_data["photo_image_url"]
        
        images.append(image_path)       
        score = "{:.5f}".format(doc_scores[doc_idx])

        labels.append (f"""
                     Photo title: {photo_data["description_final"]}   <br/>
                     Distance: {score}
            
                     """)
        
    ipyplot.plot_images(images=images, labels=labels, img_width=200)

        

# More Examples

In [70]:
find_results( "Two dogs playing in the snow", explain=False, k =5)

['two', 'dogs', 'playing', 'in', 'the', 'snow']


In [71]:
find_results( "boy and girl on a beach")

['boy', 'and', 'girl', 'on', 'a', 'beach']


In [72]:
find_results( "image of a man in a desert")

['image', 'of', 'a', 'man', 'in', 'a', 'desert']


In [73]:
sample_queries = [

"person on top of mountain"
, "picture of a man in a desert"
, "person in a desert"    

, "the boy and girl on a beach"
, "children in beach"    

, "Two dogs playing in the snow"

, "light at the end of the tunnel"
, "seven wonders of the world"
    
    
, "water droplets on a leaf"
    
, "ripley's aquarium of canada, toronto, canada"
, "the butterfly atrium at hershey gardens"
    
, "salar de uyuni uyuni bolivia"
, "沙漠青蛙 沙漠青蛙" #(desert frog)
, "por do sol no mar"
, "conhece te a ti mesmo" #	 ( Greek for know thyself)


, "there is no planet b"


, "nova scotia duck tolling retriever"
    
]

In [74]:
@interact
def interact_find_results(query=sample_queries, k =5):
    find_results( query, k =k)

interactive(children=(Dropdown(description='query', options=('person on top of mountain', 'picture of a man in…

1) How the `person on top of mountain` compare to the BM25 approach? Is it better or about the same?

2) How do the results of `picture of a man in a desert` and `person in a desert` compare ? 

3) How do `the boy and girl on a beach` and `children in beach` compare ?

4) How do the top result in `Two dogs playing in the snow` compare ?
5) How do the results of in `ight at the end of the tunnel` look ?

6) Are the results of `seven wonders of the world`relevnat ?

# Performance

In [75]:
%%timeit
search_query = "Two dogs playing in the snow"
k=5
text_features = model.encode(search_query, normalize_embeddings=True)
doc_scores = text_features @ corpus.T

top_items = doc_scores.argsort()[-k:][::-1]


19.6 ms ± 587 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In a BM-25 approach, we were able to get the results in ~3 ms